In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from google.colab import drive
import librosa
import h5py
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorflow_version 2.x
import datetime, os
drive.mount('/content/drive')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
TensorFlow is already loaded. Please restart the runtime to change versions.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with h5py.File('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/4channel_singlefile.hdf5', 'r') as hf:
    clear_train = hf.get('clear_timit_train')
    mixture_train = hf.get('mixture_timit_train')

    clear_train = np.array(clear_train)
    mixture_train = np.array(mixture_train)

In [0]:
noisyMean     = np.mean(mixture_train);
noisyStd      = np.std(mixture_train);
print(str(noisyMean))
print(str(noisyStd))
mixture_train = (mixture_train - noisyMean)/noisyStd;
cleanMean     = np.mean(clear_train);
cleanStd      = np.std(clear_train);
print(str(cleanMean))
print(str(cleanStd))
clear_train    = (clear_train - cleanMean)/cleanStd;

0.0143490728753785
0.08059500852742855
0.28596476
1.1955631


### Channels Last Approach
We will create the following structure: 
(296324,129, 8, 1)
(296324,129, 1, 1)


In [0]:
mixture_train_shaped = np.reshape(mixture_train, (mixture_train.shape[0],mixture_train.shape[1], mixture_train.shape[2], 4));
clear_train_shaped = np.reshape(clear_train, (clear_train.shape[0], clear_train.shape[1], 1, 1));
print(mixture_train_shaped.shape)
print(clear_train_shaped.shape)

(53, 129, 8, 4)
(53, 129, 1, 1)


In [0]:
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, Flatten, Dense, Reshape, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras import Model, Sequential
input_img = tf.keras.layers.Input(shape=(129,8,4))
x = tf.keras.layers.Conv2D(18, (9, 8), strides=(1, 100), padding='same')(input_img)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)

for i in range(0,5):
	x = tf.keras.layers.Conv2D(30, (5, 1), strides=(1, 100), padding='same')(x)
	x = tf.keras.layers.BatchNormalization()(x)
	x = tf.keras.layers.ReLU()(x)
	x = tf.keras.layers.Conv2D(8, (9, 1), strides=(1, 100), padding='same')(x)
	x = tf.keras.layers.BatchNormalization()(x)
	x = tf.keras.layers.ReLU()(x)
	x = tf.keras.layers.Conv2D(18, (9, 1), strides=(1, 100), padding='same')(x)
	x = tf.keras.layers.BatchNormalization()(x)
	x = tf.keras.layers.ReLU()(x)
	x = tf.keras.layers.Conv2D(30, (5, 1), strides=(1, 100), padding='same')(x)
	x = tf.keras.layers.BatchNormalization()(x)
	x = tf.keras.layers.ReLU()(x)

decoded = tf.keras.layers.Conv2D(1, (129, 1), strides=(1, 100), padding='same')(x)

model = Model(inputs=input_img, outputs=decoded)

optimizer = keras.optimizers.Adam(learning_rate=0.0015,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    amsgrad=True,
    name='Adam')

model.compile(optimizer=optimizer, loss='mean_squared_error')
model.summary()

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/denoiser_cnn_log_mel_generator.h5')

In [0]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True, baseline=0.14)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='batch')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/4channel_single.h5', 
                                                         monitor='val_loss', save_best_only=True)
model.fit(mixture_train_shaped, clear_train_shaped,
         validation_split=0.1,
         epochs=5000,
         batch_size=256, 
         shuffle=True,
         callbacks=[early_stopping_callback, tensorboard_callback, checkpoint_callback]
        )
model.save('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/4channel_single.h5') 

Train on 47 samples, validate on 6 samples
Epoch 1/5000
47/47 [==============================] - 11s 227ms/sample - loss: 1.5254 - val_loss: 0.0531
Epoch 2/5000
47/47 [==============================] - 0s 9ms/sample - loss: 1.3747 - val_loss: 0.0528
Epoch 3/5000
47/47 [==============================] - 0s 719us/sample - loss: 1.0994 - val_loss: 0.0533
Epoch 4/5000
47/47 [==============================] - 0s 647us/sample - loss: 0.9324 - val_loss: 0.0540
Epoch 5/5000
47/47 [==============================] - 0s 671us/sample - loss: 0.8851 - val_loss: 0.0539
Epoch 6/5000
47/47 [==============================] - 0s 565us/sample - loss: 0.7841 - val_loss: 0.0529
Epoch 7/5000
47/47 [==============================] - 0s 9ms/sample - loss: 0.6291 - val_loss: 0.0513
Epoch 8/5000
47/47 [==============================] - 0s 9ms/sample - loss: 0.5185 - val_loss: 0.0494
Epoch 9/5000
47/47 [==============================] - 0s 9ms/sample - loss: 0.4665 - val_loss: 0.0478
Epoch 10/5000
47/47 [=======

### Loading an Existing Model to Continue Training

In [0]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


### Code Evaluation
Testing the success of the algorithm with the dataset.

In [0]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/4channel_single.h5')

# Show the model architecture
# model.summary()

## Testing the Model
Here I will be continuing to improve the model

In [0]:
counter = 0
FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
Overlap = round(0.75 * FrameSize)
FFTSize = FrameSize  # FFT window size=FRAMESIZE
FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
NumSegments = 8
noisyMean = 0.0143
noisyStd  = 0.0805
cleanStd  = 0.286
cleanMean = 1.1955631

def convert_to_audio(outname,stft_magnitude,stft_phase,noisyAudio,fs=8000,):
    # Normal Case:
    noisyMean = 0.0143
    noisyStd  = 0.0805
    cleanStd  = 0.286
    cleanMean = 1.1955631

    # Converting the data back to human language
    FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
    Overlap   = round(0.75 * FrameSize)
    FFTSize   = FrameSize  # FFT window size=FRAMESIZE
    FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
    NumSegments = 8
    if noisyAudio == True:
      stft_magnitude_nonNormal = noisyStd*stft_magnitude+noisyMean
    else:
      stft_magnitude_nonNormal = cleanStd*stft_magnitude+cleanMean
    final_STFT = stft_magnitude_nonNormal*np.exp(1j*stft_phase)
    # Audio Converter
    converted_audio = librosa.istft(final_STFT, hop_length=Overlap, win_length=FFTSize,
                  window=scipy.signal.hamming(FrameSize,sym=False))
    wavfile.write(outname, rate=fs, data=converted_audio)

def stft_generator(filename,noisyAudio,fs=8000):
    FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
    Overlap = round(0.75 * FrameSize)
    FFTSize = FrameSize  # FFT window size=FRAMESIZE
    FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
    NumSegments = 8

    # Reading the audio data
    audio_data, sr = librosa.load(filename, mono=False, sr=fs)  # if sr=None to read raw sample_rate

    if noisyAudio == False:
        stft_segments= librosa.stft(audio_data, n_fft=FrameSize, hop_length=Overlap, win_length=FFTSize,
                                              window=scipy.signal.hamming(FrameSize,sym=False))
        # Phase
        audio_phase = np.angle(stft_segments)
        # Magnitude matrix
        stft_segments = np.transpose(abs(stft_segments))
    else:
        counter = 0
        for audio in audio_data:
            channelNumber = 4
            audio = np.nan_to_num(audio)
            noisySTFT = librosa.stft(audio, n_fft=FrameSize, hop_length=Overlap, win_length=FFTSize,
                                         window=scipy.signal.hamming(FrameSize,sym=False))
            audio_phase = np.angle(noisySTFT)
            # Magnitude matrix
            noisySTFT =np.abs(noisySTFT)
            new_noisy_STFT = np.concatenate((noisySTFT[:,0:NumSegments-1], noisySTFT), axis=1)
            stft_segments = np.zeros((new_noisy_STFT.shape[1] - NumSegments + 1, FrequencyBins, NumSegments,channelNumber))
            for index in range(0, new_noisy_STFT.shape[1] - NumSegments + 1):
                stft_segments[index,:,:,counter] = new_noisy_STFT[:, index:index+NumSegments]
            counter = counter + 1
    return {'stft': stft_segments, 'phase': audio_phase}

In [12]:
# Creating the magnitude and phase spectrums of noisy signal
from scipy.io import wavfile
import scipy
stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/4Channel/mixture_494_n2.wav',True)
clean_stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/4Channel/perfect494.wav',False)
absSTFT = stft_segments['stft']
noisyPhase = stft_segments['phase']
absSTFT_normalized = (absSTFT - noisyMean)/noisyStd
cleanPhase = clean_stft_segments['phase']
clean_stft_segments = clean_stft_segments['stft']
clean_absSTFT_normalized = (clean_stft_segments - cleanMean)/cleanStd
print(absSTFT.shape)
print(clean_stft_segments.shape)

(53, 129, 8, 4)
(53, 129)


In [13]:
noisySTFT = np.reshape(absSTFT_normalized, (absSTFT_normalized.shape[0],absSTFT_normalized.shape[1], absSTFT_normalized.shape[2], 4));
cleanSTFT = np.reshape(clean_absSTFT_normalized, (clean_absSTFT_normalized.shape[0], clean_absSTFT_normalized.shape[1], 1, 1));
print(noisySTFT.shape)
print(cleanSTFT.shape)
cleanedup_audio = model.predict(noisySTFT)
print(cleanedup_audio.shape)

(53, 129, 8, 4)
(53, 129, 1, 1)
(53, 129, 1, 1)


In [14]:
print('\n# Evaluate on test data')
results = model.evaluate(noisySTFT, cleanSTFT,verbose=1)
print('test loss, test acc:', results)


# Evaluate on test data
53/53 [==============================] - 1s 11ms/sample - loss: 23.2875
test loss, test acc: 23.2875197968393


In [15]:
final_cleanedup_audio = np.reshape(cleanedup_audio, (noisySTFT.shape[0], noisySTFT.shape[1]));
print(final_cleanedup_audio.shape)
print(noisyPhase.shape)

final_cleanSTFT= np.reshape(cleanSTFT, (clean_stft_segments.shape[0], clean_stft_segments.shape[1]));
print(final_cleanSTFT.shape)
print(cleanPhase.shape)

(53, 129)
(129, 53)
(53, 129)
(129, 53)


### Write to the file
The code below converts the cleaned file to the drive.


In [0]:
convert_to_audio('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/4Channel/mixture_494_n2_Cleaned.wav',
                 np.transpose(final_cleanedup_audio),noisyPhase,True)
# convert_to_audio('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/4Channel/perfect0_8K.wav',
#                  np.transpose(final_cleanSTFT),cleanPhase,False)

# PLAYGROUND FOR GROUND TRUTH

In [0]:
mse = (np.square(final_cleanedup_audio - final_cleanSTFT)).mean()
print(mse)
stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145_n1.wav',False)
absSTFT = stft_segments['stft']
noisyPhase = stft_segments['phase']
absSTFT_normalized = (absSTFT - noisyMean)/noisyStd
print(absSTFT_normalized.shape)
mse = (np.square(absSTFT_normalized - final_cleanSTFT)).mean()
print(mse)

In [0]:
# Creating the magnitude and phase spectrums of noisy signal
from scipy.io import wavfile
import scipy
stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145.wav',True)
clean_stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145.wav',False)
absSTFT = stft_segments['stft']
noisyPhase = stft_segments['phase']
absSTFT_normalized = (absSTFT - cleanMean)/cleanStd
cleanPhase = clean_stft_segments['phase']
clean_stft_segments = clean_stft_segments['stft']
clean_absSTFT_normalized = (clean_stft_segments - cleanMean)/cleanStd
print(absSTFT.shape)
print(clean_stft_segments.shape)

(129, 8, 325)
(129, 325)


### Demo Code


In [0]:
for i in range(8):
  print(sum(np.ravel(absSTFT[:,i,:])-np.ravel(clean_stft_segments[:,:])))
cleanedup_audio = new_model.predict(noisySTFT)
print(absSTFT.shape)
print(clean_stft_segments.shape)


-5.521531762433824
-4.648348442810402
-3.690907558631352
-2.8582976624525145
-2.064867725527165
-1.2853483124349623
-0.4730873316739235
0.0
(129, 8, 325)
(129, 325)


In [0]:
noisySTFT = np.reshape(absSTFT_normalized, (absSTFT_normalized.shape[2],absSTFT_normalized.shape[0], absSTFT_normalized.shape[1], 1));
cleanSTFT = np.reshape(clean_absSTFT_normalized, (clean_absSTFT_normalized.shape[1], clean_absSTFT_normalized.shape[0], 1, 1));
noisySTFT = np.ravel(noisySTFT)
print(noisySTFT.shape)
noisySTFT = np.reshape(noisySTFT, (absSTFT_normalized.shape[0],absSTFT_normalized.shape[1], absSTFT_normalized.shape[2]));
print(noisySTFT.shape)
cleanSTFT = np.ravel(cleanSTFT)
print(cleanSTFT.shape)
cleanSTFT = np.reshape(cleanSTFT, (clean_absSTFT_normalized.shape[0], clean_absSTFT_normalized.shape[1]));
print(cleanSTFT.shape)

for i in range(8):
  print(sum(np.ravel(noisySTFT[:,i,:])-np.ravel(cleanSTFT[:,:])))
# cleanedup_audio = new_model.predict(noisySTFT)
# print(noisySTFT.shape)
# print(cleanSTFT.shape)
# print(cleanedup_audio.shape)
# print(noisyPhase.shape)

(335400,)
(129, 8, 325)
(41925,)
(129, 325)
-6.570148158180036
-5.531115976335604
-4.391822693334616
-3.4010702847510186
-2.456939516357768
-1.529361379701967
-0.5628228761593751
0.00012072501257150217


**Evaluation in numbers**


In [0]:
print('\n# Evaluate on test data')
results = new_model.evaluate(noisySTFT, cleanSTFT,verbose=1)
print('test loss, test acc:', results)


# Evaluate on test data
325/325 [==============================] - 0s 269us/sample - loss: 0.7077
test loss, test acc: 0.707741042226553


In [0]:
final_audio = np.transpose(np.squeeze(cleanSTFT))
print(final_audio[1:3,1:3])
print(clean_absSTFT_normalized[1:3,1:3])
print(final_audio.shape)
print(clean_absSTFT_normalized.shape)
print(cleanPhase.shape)

[[-0.28905934 -0.26405573]
 [-0.26985237 -0.3025639 ]]
[[-0.29344794 -0.2982164 ]
 [-0.3020195  -0.30220264]]
(129, 325)
(129, 325)
(129, 325)


In [0]:
convert_to_audio('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145furka_cleanedv2.wav',
                 clean_absSTFT_normalized,cleanPhase,False)